In [ ]:
! sudo pip3 install dgl-cu101

     |████████████████████████████████| 36.2MB 87kB/s 


In [ ]:
! nvidia-smi

Mon Jun 21 00:38:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! sudo pip3 install dglke

     |████████████████████████████████| 81kB 5.1MB/s 


In [ ]:
! git clone https://github.com/gnn4dr/DRKG.git

Cloning into 'DRKG'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 283 (delta 48), reused 81 (delta 45), pack-reused 188
Receiving objects: 100% (283/283), 1.39 MiB | 4.33 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
import pandas as pd
import numpy as np
import dgl
import sys
sys.path.insert(1, 'DRKG/utils')
from utils import download_and_extract
download_and_extract()
drkg_file = '../data/drkg.tsv'
df = pd.read_csv(drkg_file, sep="\t")
triplets = df.values.tolist()

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


Download finished. Unzipping the file...


In [ ]:
! wget https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz


--2021-06-21 00:41:14--  https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz
Resolving dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)... 52.92.146.194
Connecting to dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)|52.92.146.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216650245 (207M) [application/x-tar]
Saving to: ‘drkg.tar.gz’

drkg.tar.gz         100%[===================>] 206.61M  65.1MB/s    in 3.3s    

2021-06-21 00:41:17 (62.2 MB/s) - ‘drkg.tar.gz’ saved [216650245/216650245]



In [ ]:
! tar -xf drkg.tar.gz

In [ ]:
! ls embed/

DRKG_TransE_l2_entity.npy    mol_contextpred.npy  mol_masking.npy
DRKG_TransE_l2_relation.npy  mol_edgepred.npy	  Readme.md
entities.tsv		     mol_infomax.npy	  relations.tsv


In [ ]:
! ls  DRKG/drug_repurpose/infer_drug.tsv

coronavirus-related-host-genes.tsv	   covid19-host-genes.tsv
COVID19_clinical_trial_drugs.tsv	   infer_drug.tsv
COVID-19_drug_repurposing.ipynb		   Readme.md
COVID-19_drug_repurposing_via_genes.ipynb


In [ ]:
! ls embed

DRKG_TransE_l2_entity.npy    mol_contextpred.npy  mol_masking.npy
DRKG_TransE_l2_relation.npy  mol_edgepred.npy	  Readme.md
entities.tsv		     mol_infomax.npy	  relations.tsv


In [ ]:
print(len(triplets))

5874260


In [ ]:

COV_disease_list = [
'Disease::SARS-CoV2 E',
'Disease::SARS-CoV2 M',
'Disease::SARS-CoV2 N',
'Disease::SARS-CoV2 Spike',
'Disease::SARS-CoV2 nsp1',
'Disease::SARS-CoV2 nsp10',
'Disease::SARS-CoV2 nsp11',
'Disease::SARS-CoV2 nsp12',
'Disease::SARS-CoV2 nsp13',
'Disease::SARS-CoV2 nsp14',
'Disease::SARS-CoV2 nsp15',
'Disease::SARS-CoV2 nsp2',
'Disease::SARS-CoV2 nsp4',
'Disease::SARS-CoV2 nsp5',
'Disease::SARS-CoV2 nsp5_C145A',
'Disease::SARS-CoV2 nsp6',
'Disease::SARS-CoV2 nsp7',
'Disease::SARS-CoV2 nsp8',
'Disease::SARS-CoV2 nsp9',
'Disease::SARS-CoV2 orf10',
'Disease::SARS-CoV2 orf3a',
'Disease::SARS-CoV2 orf3b',
'Disease::SARS-CoV2 orf6',
'Disease::SARS-CoV2 orf7a',
'Disease::SARS-CoV2 orf8',
'Disease::SARS-CoV2 orf9b',
'Disease::SARS-CoV2 orf9c',
'Disease::MESH:D045169',
'Disease::MESH:D045473',
'Disease::MESH:D001351',
'Disease::MESH:D065207',
'Disease::MESH:D028941',
'Disease::MESH:D058957',
'Disease::MESH:D006517'
]

In [ ]:
entity_emb = np.load('./embed/DRKG_TransE_l2_entity.npy')
print(entity_emb.shape)

(97238, 400)


In [ ]:
import csv

# Load entity file
drug_list = []
with open("./DRKG/drug_repurpose/infer_drug.tsv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
    for row_val in reader:
        drug_list.append(row_val['drug'])

In [ ]:
len(drug_list)

8104

In [ ]:
treatment = ['Hetionet::CtD::Compound:Disease','GNBR::T::Compound:Disease']

In [ ]:
entity_idmap_file = 'embed/entities.tsv'
relation_idmap_file = 'embed/relations.tsv'

In [ ]:
# Get drugname/disease name to entity ID mappings
entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
drug_ids = []
disease_ids = []
for drug in drug_list:
    drug_ids.append(entity_map[drug])
    
for disease in COV_disease_list:
    disease_ids.append(entity_map[disease])

treatment_rid = [relation_map[treat]  for treat in treatment]

In [ ]:
# Load embeddings
import torch as th
entity_emb = np.load('embed/DRKG_TransE_l2_entity.npy')
rel_emb = np.load('embed/DRKG_TransE_l2_relation.npy')

drug_ids = th.tensor(drug_ids).long()
disease_ids = th.tensor(disease_ids).long()
treatment_rid = th.tensor(treatment_rid)

drug_emb = th.tensor(entity_emb[drug_ids])
treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

In [ ]:
import torch.nn.functional as fn

gamma=12.0
def transE_l2(head, rel, tail):
    score = head + rel - tail
    return gamma - th.norm(score, p=2, dim=-1)

scores_per_disease = []
dids = []
for rid in range(len(treatment_embs)):
    treatment_emb=treatment_embs[rid]
    for disease_id in disease_ids:
        disease_emb = entity_emb[disease_id]
        score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
        scores_per_disease.append(score)
        dids.append(drug_ids)
scores = th.cat(scores_per_disease)
dids = th.cat(dids)

In [ ]:
# sort scores in decending order
idx = th.flip(th.argsort(scores), dims=[0])
scores = scores[idx].numpy()
dids = dids[idx].numpy()

In [ ]:

_, unique_indices = np.unique(dids, return_index=True)
topk=100
topk_indices = np.sort(unique_indices)[:topk]
proposed_dids = dids[topk_indices]
proposed_scores = scores[topk_indices]

In [ ]:
for i in range(topk):
    drug = int(proposed_dids[i])
    score = proposed_scores[i]
    
    print("{}\t{}".format(entity_id_map[drug], score))

Compound::DB00811	-0.21416784822940826
Compound::DB00993	-0.8350892663002014
Compound::DB00635	-0.8974801898002625
Compound::DB01082	-0.9854875802993774
Compound::DB01234	-0.9984006881713867
Compound::DB00982	-1.0160715579986572
Compound::DB00563	-1.0189464092254639
Compound::DB00290	-1.0641038417816162
Compound::DB01394	-1.080674648284912
Compound::DB01222	-1.084547519683838
Compound::DB00415	-1.0853980779647827
Compound::DB01004	-1.096668004989624
Compound::DB00860	-1.1004775762557983
Compound::DB00681	-1.1011559963226318
Compound::DB00688	-1.1256868839263916
Compound::DB00624	-1.1428287029266357
Compound::DB00959	-1.1618403196334839
Compound::DB00115	-1.1868137121200562
Compound::DB00091	-1.1906721591949463
Compound::DB01024	-1.2051165103912354
Compound::DB00741	-1.2147057056427002
Compound::DB00441	-1.2320444583892822
Compound::DB00158	-1.2346539497375488
Compound::DB00499	-1.2525147199630737
Compound::DB00929	-1.2730510234832764
Compound::DB00770	-1.2825534343719482
Compound::DB01

In [ ]:

clinical_drugs_file = 'DRKG/drug_repurpose/COVID19_clinical_trial_drugs.tsv'
clinical_drug_map = {}
with open(clinical_drugs_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['id', 'drug_name','drug_id'])
    for row_val in reader:
        clinical_drug_map[row_val['drug_id']] = row_val['drug_name']
        
for i in range(topk):
    drug = entity_id_map[int(proposed_dids[i])][10:17]
    if clinical_drug_map.get(drug, None) is not None:
        score = proposed_scores[i]
        print("[{}]\t{}\t{}".format(i, clinical_drug_map[drug],score , proposed_scores[i]))

[0]	Ribavirin	-0.21416784822940826
[4]	Dexamethasone	-0.9984006881713867
[8]	Colchicine	-1.080674648284912
[16]	Methylprednisolone	-1.1618403196334839
[49]	Oseltamivir	-1.3885014057159424
[87]	Deferoxamine	-1.5130667686462402


In [ ]:
len(clinical_drug_map)

32

In [ ]:
from google.colab import files

files.download('edge_pair_jaccard_scores_sorted_jacard.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>